<a href="https://colab.research.google.com/github/mohammadRahimi1993/Exercise/blob/main/NodeClassification_DGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Node Classification with DGL**

1- **Install DGL And Torch**

In [1]:
# https://colab.research.google.com/github/dmlc/dgl/blob/master/notebooks/sparse/quickstart.ipynb#scrollTo=19UZd7wyWzpT
!pip install torch==2.0.0+cpu torchvision==0.15.1+cpu torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cpu > /dev/null
!pip install dgl==1.1.0 -f https://data.dgl.ai/wheels/repo.html > /dev/null

2- **Check dgl and torch**

In [2]:
import os
import torch

os.environ["TORCH"] = torch.__version__
os.environ["DGLBACKEND"] = "pytorch"

try:
  import dgl.sparse as dglsp
  installed = True
except ImportError:
  installed = False

print("installed" if installed else "not installed!")

installed


3- **Import library**

In [3]:
import torch
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

4- **Download CoraGraphDataset**

In [4]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of Categories: {dataset.num_classes}")
print(f"Number of Labels: {dataset.num_labels}")

Extracting file to /root/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of Categories: 7
Number of Labels: 7


/usr/local/lib/python3.11/dist-packages/dgl/data/utils.py:354: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn(


6- **Defining a Graph Convolutional Network (GCN)**

In [7]:
from dgl.nn import GraphConv

class GCN(nn.Module):
  def __init__(self , in_feats, h_feats, num_classes ):
    super(GCN, self).__init__()
    self.conv1 = GraphConv(in_feats, h_feats)
    self.conv2 = GraphConv(h_feats, num_classes)

  def forward(self, g, in_feat):
    h = self.conv1(g, in_feat)
    h = F.relu(h)
    h = self.conv2(g, h)
    return h

In [8]:
g = dataset[0]

print("Edge feature")
print(g.ndata)
print("Node feature")
print(g.edata)

Edge feature
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Node feature
{}


7- **Training the GCN**

In [9]:
def train(g, model):
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  best_val_acc = 0
  best_test_acc = 0

  featurs = g.ndata["feat"]
  labels = g.ndata["label"]
  train_mask = g.ndata["train_mask"]
  val_mask = g.ndata["val_mask"]
  test_mask = g.ndata["test_mask"]

  for e in range(100):
    logits = model(g, featurs)

    # Compute prediction
    pred = logits.argmax(1)

    # Compute loss
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])

    train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
    val_acc =   (pred[val_mask] == labels[val_mask]).float().mean()
    test_acc =  (pred[test_mask] == labels[test_mask]).float().mean()

    if best_val_acc < val_acc:
      best_val_acc = val_acc
      best_val_test = test_acc

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e%5 == 0:
      print(f"In epoch {e}, loss:{loss:.3f}, val_acc:{val_acc:.3f} (best_val_acc {best_val_acc:.3f} , test_acc:{test_acc:.3f}) best_test_acc { best_val_test:.3f}")

model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

/usr/local/lib/python3.11/dist-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In epoch 0, loss:1.946, val_acc:0.130 (best_val_acc 0.130 , test_acc:0.124) best_test_acc 0.124
In epoch 5, loss:1.895, val_acc:0.422 (best_val_acc 0.422 , test_acc:0.445) best_test_acc 0.445
In epoch 10, loss:1.814, val_acc:0.394 (best_val_acc 0.448 , test_acc:0.439) best_test_acc 0.476
In epoch 15, loss:1.711, val_acc:0.498 (best_val_acc 0.498 , test_acc:0.524) best_test_acc 0.524
In epoch 20, loss:1.586, val_acc:0.618 (best_val_acc 0.618 , test_acc:0.615) best_test_acc 0.615
In epoch 25, loss:1.440, val_acc:0.668 (best_val_acc 0.668 , test_acc:0.665) best_test_acc 0.665
In epoch 30, loss:1.280, val_acc:0.690 (best_val_acc 0.690 , test_acc:0.707) best_test_acc 0.707
In epoch 35, loss:1.112, val_acc:0.708 (best_val_acc 0.708 , test_acc:0.740) best_test_acc 0.740
In epoch 40, loss:0.944, val_acc:0.732 (best_val_acc 0.732 , test_acc:0.751) best_test_acc 0.749
In epoch 45, loss:0.784, val_acc:0.742 (best_val_acc 0.742 , test_acc:0.767) best_test_acc 0.767
In epoch 50, loss:0.640, val_acc